In [5]:
import joblib
le_gu = joblib.load("../../Predict_models/M2/M2_ENCODER/행정구라벨인코더.pkl")
le_target = joblib.load("../../Predict_models/M2/M2_ENCODER/타겟변수라벨인코더.pkl")
le_target.classes_

c:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array(['공급과다', '공급다소부족', '공급부족', '공급절대부족', '공급평균'], dtype=object)

In [7]:
import pandas as pd
pd.DataFrame([[1],[2],[3]])

,0
0,1
1,2
2,3


In [11]:
import pymysql
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
from itertools import product

def DBMS_last_year_one_week_M2() :
    '''내일을 기준으로 작년 1주일치 생활인구수와 대여량을 DataFrame으로 반환하는 함수'''
    load_dotenv()
    host = os.getenv('host')
    user = os.getenv('user')
    password = os.getenv('password')
    database = os.getenv('database')
    port = int(os.getenv('port', 3306))

    # 현재 시각
    now = datetime.now()

    # 1주일치 시간 list
    this_week = [now + timedelta(days=i) for i in range(8)]

    # 1년 전 1주일치 데이터
    dt_1y = [day.replace(year=day.year - 1) for day in this_week]

    total_last_week = pd.DataFrame(columns=['연도','월','일','시','행정구','일시','총생활인구수_1년전','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','대여량_1년전'])
    for data in dt_1y :
        query =f"""
            SELECT *
            FROM basic_data
            WHERE year = {data.year}
                AND month = {data.month}
                AND day = {data.day}
                """

        try:
            connection = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database,
                port=port,
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            )
            with connection.cursor() as cursor:
                cursor.execute(query)
                data = cursor.fetchall()
                whole_data = pd.DataFrame(data)
                whole_data.columns = ['연도','월','일','시','행정구','일시','총생활인구수_1년전','계절','출퇴근시간여부','주말구분','강수','기온','습도','풍속','대여량_1년전']
                total_last_week = pd.concat((total_last_week,whole_data),axis=0)
        except Exception as e:
            print('에러 발생:', e)
        finally:
            try:
                connection.close()
            except:
                pass
    return total_last_week

data = DBMS_last_year_one_week_M2()

def fill_missing_timeseries(data=data, days=8, key_cols=None, num_cols=None):
    """
    data: DataFrame, 기준이 되는 원본 데이터 (key_cols 모두 포함)
    days: int, 몇 일간의 데이터를 보장할지 (기본값 8일)
    key_cols: list, 시계열 Key가 되는 컬럼 (예: ['월','일','시','행정구'])
    num_cols: list, 결측값 채울 대상 컬럼. None이면 모든 컬럼 채움

    반환값: 누락 구간이 보간된, 중복 없는 DataFrame
    """
    if key_cols is None:
        key_cols = ['월', '일', '시', '행정구']
    if num_cols is None:
        num_cols = [col for col in data.columns if col not in key_cols]
    for col in key_cols:
        data[col] = data[col].astype(str)
    
    # 1. 전체 조합 만들기
    now = datetime.now().date()
    dates = pd.date_range(now, now + timedelta(days=days-1))   # 8일
    hours = list(range(24))
    gus = data['행정구'].unique().tolist()

    full = []
    for dt in dates:
        for h in hours:
            for g in gus:
                full.append({
                    '월': str(dt.month),
                    '일': str(dt.day),
                    '시': str(h),
                    '행정구': str(g)
                })
    full_df = pd.DataFrame(full)

    # 2. 누락행 탐색 및 합치기
    merged = full_df.merge(data, on=key_cols, how='left', indicator=True)
    missing = merged[merged['_merge']=='left_only']
    df_all = pd.concat([data, missing.drop('_merge', axis=1)], ignore_index=True)
    df_all = df_all.sort_values(key_cols).reset_index(drop=True)

    # 3. ffill, bfill 결측치 채우기 (numeric 및 object 모두 적용 가능)
    for col in num_cols:
        df_all[col] = df_all.groupby(['행정구', '시'])[col].ffill()
        df_all[col] = df_all.groupby(['행정구', '시'])[col].bfill()

    # 4. 중복제거 및 정렬
    df_all = df_all.drop_duplicates(subset=key_cols, keep='first')
    df_all = df_all.sort_values(key_cols).reset_index(drop=True)
    return df_all

# 사용 예시
# df_result = fill_missing_timeseries(data, days=8, key_cols=['월','일','시','행정구'])


if __name__ == "__main__" :
    check = fill_missing_timeseries()
    print(check)

        연도  월   일  시  행정구                  일시 총생활인구수_1년전  계절 출퇴근시간여부 주말구분  \
0     2024  7  11  0  강남구 2024-07-11 00:00:00     701439  여름      기타    0   
1     2024  7  11  0  강동구 2024-07-11 00:00:00     535574  여름      기타    0   
2     2024  7  11  0  강북구 2024-07-11 00:00:00     285612  여름      기타    0   
3     2024  7  11  0  강서구 2024-07-11 00:00:00     546611  여름      기타    0   
4     2024  7  11  0  관악구 2024-07-11 00:00:00     495542  여름      기타    0   
...    ... ..  .. ..  ...                 ...        ...  ..     ...  ...   
4795  2024  7  18  9  용산구 2024-07-16 09:00:00     293613  여름    출근시간    0   
4796  2024  7  18  9  은평구 2024-07-18 09:00:00     382284  여름    출근시간    0   
4797  2024  7  18  9  종로구 2024-07-18 09:00:00     330689  여름    출근시간    0   
4798  2024  7  18  9   중구 2024-07-18 09:00:00     389344  여름    출근시간    0   
4799  2024  7  18  9  중랑구 2024-07-18 09:00:00     305389  여름    출근시간    0   

       강수    기온    습도   풍속 대여량_1년전  
0     0.0  25.8  76.0  2.4     130  
1

In [10]:
import requests
import pandas as pd

def get_hourly_weather_seoul_openmeteo():
    SEOUL_DISTRICTS = [
        {"name": "강남구", "lat": 37.5172, "lon": 127.0473},
        {"name": "강동구", "lat": 37.5301, "lon": 127.1238},
        {"name": "강북구", "lat": 37.6396, "lon": 127.0256},
        {"name": "강서구", "lat": 37.5509, "lon": 126.8495},
        {"name": "관악구", "lat": 37.4784, "lon": 126.9516},
        {"name": "광진구", "lat": 37.5384, "lon": 127.0823},
        {"name": "구로구", "lat": 37.4955, "lon": 126.8878},
        {"name": "금천구", "lat": 37.4604, "lon": 126.9006},
        {"name": "노원구", "lat": 37.6542, "lon": 127.0568},
        {"name": "도봉구", "lat": 37.6688, "lon": 127.0472},
        {"name": "동대문구", "lat": 37.5744, "lon": 127.0396},
        {"name": "동작구", "lat": 37.5124, "lon": 126.9392},
        {"name": "마포구", "lat": 37.5663, "lon": 126.9014},
        {"name": "서대문구", "lat": 37.5791, "lon": 126.9368},
        {"name": "서초구", "lat": 37.4836, "lon": 127.0327},
        {"name": "성동구", "lat": 37.5633, "lon": 127.0364},
        {"name": "성북구", "lat": 37.5894, "lon": 127.0167},
        {"name": "송파구", "lat": 37.5146, "lon": 127.1056},
        {"name": "양천구", "lat": 37.5179, "lon": 126.8666},
        {"name": "영등포구", "lat": 37.5264, "lon": 126.8962},
        {"name": "용산구", "lat": 37.5323, "lon": 126.9909},
        {"name": "은평구", "lat": 37.6027, "lon": 126.9291},
        {"name": "종로구", "lat": 37.5735, "lon": 126.9790},
        {"name": "중구", "lat": 37.5636, "lon": 126.9977},
        {"name": "중랑구", "lat": 37.6065, "lon": 127.0927}
    ]

    result_rows = []
    for gu in SEOUL_DISTRICTS:
        latitude, longitude, name = gu["lat"], gu["lon"], gu["name"]
        url = (
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={latitude}&longitude={longitude}"
            "&hourly=temperature_2m,precipitation,weathercode,relative_humidity_2m,wind_speed_10m"
            "&forecast_days=8&timezone=Asia%2FSeoul"
        )
        resp = requests.get(url)
        if resp.status_code == 200:
            json_data = resp.json()
            if 'hourly' not in json_data:
                print(f"[경고] {name} 데이터에 'hourly' 없음. 응답: {json_data}")
                continue
            data = json_data['hourly']
            for t, temp, rain, code, rh, wind in zip(
                data['time'],
                data['temperature_2m'],
                data['precipitation'],
                data['weathercode'],
                data['relative_humidity_2m'],
                data['wind_speed_10m'],
            ):
                result_rows.append({
                    "행정구": name,
                    "일시": t,
                    "강수": rain,
                    "습도": rh,
                    "풍속": wind,
                    "기온": temp,
                })
        else:
            print(f"[오류] {name} API 호출 실패: {resp.status_code}")
    df = pd.DataFrame(result_rows)
    return df


if __name__ == "__main__" :
    weather = get_hourly_weather_seoul_openmeteo()
    print(weather)

      행정구                일시   강수  습도    풍속    기온
0     강남구  2025-07-11T00:00  0.0  71  12.7  23.0
1     강남구  2025-07-11T01:00  0.0  74  11.3  22.1
2     강남구  2025-07-11T02:00  0.0  78  11.0  21.5
3     강남구  2025-07-11T03:00  0.0  79  11.4  21.2
4     강남구  2025-07-11T04:00  0.0  82  10.6  20.7
...   ...               ...  ...  ..   ...   ...
4795  중랑구  2025-07-18T19:00  2.1  87   6.4  25.4
4796  중랑구  2025-07-18T20:00  2.1  88   6.3  25.3
4797  중랑구  2025-07-18T21:00  2.1  88   6.2  25.2
4798  중랑구  2025-07-18T22:00  0.0  73   3.3  27.4
4799  중랑구  2025-07-18T23:00  0.0  75   2.9  27.0

[4800 rows x 6 columns]


In [ ]:
check = fill_missing_timeseries()# _M2 결과
weather = get_hourly_weather_seoul_openmeteo()
check.drop(['연도','강수','습도','풍속','기온'],axis=1, inplace=True)
weather_df = weather
weather_df['일시'] = pd.to_datetime(weather_df['일시'])
weather_df['연도'] = weather_df['일시'].dt.year.astype(str)
weather_df['월'] = weather_df['일시'].dt.month.astype(str)
weather_df['일'] = weather_df['일시'].dt.day.astype(str)
weather_df['시'] = weather_df['일시'].dt.hour.astype(str)
weather_df.drop(['일시'],axis=1, inplace=True)
df_with_weather = pd.merge(check,weather_df, on=['행정구','월','일','시'])

,연도,월,일,시,행정구,일시,총생활인구수_1년전,계절,출퇴근시간여부,주말구분,강수,기온,습도,풍속,대여량_1년전
0,2024,7,11,0,강남구,2024-07-11 00:00:00,701439,여름,기타,0,0.0,25.8,76.0,2.4,130
1,2024,7,11,0,강동구,2024-07-11 00:00:00,535574,여름,기타,0,0.0,26.3,76.0,1.3,126
2,2024,7,11,0,강북구,2024-07-11 00:00:00,285612,여름,기타,0,0.0,28.1,71.0,0.8,68
3,2024,7,11,0,강서구,2024-07-11 00:00:00,546611,여름,기타,0,0.0,25.2,78.0,2.3,292
4,2024,7,11,0,관악구,2024-07-11 00:00:00,495542,여름,기타,0,0.0,25.1,79.0,1.3,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,2024,7,18,9,용산구,2024-07-16 09:00:00,293613,여름,출근시간,0,0.0,24.9,88.0,2.5,106
4796,2024,7,18,9,은평구,2024-07-18 09:00:00,382284,여름,출근시간,0,0.0,23.9,97.0,0.1,2
4797,2024,7,18,9,종로구,2024-07-18 09:00:00,330689,여름,출근시간,0,0.0,24.7,94.0,1.6,13
4798,2024,7,18,9,중구,2024-07-18 09:00:00,389344,여름,출근시간,0,0.0,24.7,94.0,1.6,8


In [ ]:
check.drop(['연도','강수','습도','풍속','기온'],axis=1, inplace=True)
weather_df = weather
weather_df['일시'] = pd.to_datetime(weather_df['일시'])
weather_df['연도'] = weather_df['일시'].dt.year.astype(str)
weather_df['월'] = weather_df['일시'].dt.month.astype(str)
weather_df['일'] = weather_df['일시'].dt.day.astype(str)
weather_df['시'] = weather_df['일시'].dt.hour.astype(str)
weather_df.drop(['일시'],axis=1, inplace=True)
df_with_weather = pd.merge(check,weather_df, on=['행정구','월','일','시'])

,월,일,시,행정구,일시,총생활인구수_1년전,계절,출퇴근시간여부,주말구분,대여량_1년전
0,7,11,0,강남구,2024-07-11 00:00:00,701439,여름,기타,0,130
1,7,11,0,강동구,2024-07-11 00:00:00,535574,여름,기타,0,126
2,7,11,0,강북구,2024-07-11 00:00:00,285612,여름,기타,0,68
3,7,11,0,강서구,2024-07-11 00:00:00,546611,여름,기타,0,292
4,7,11,0,관악구,2024-07-11 00:00:00,495542,여름,기타,0,117
...,...,...,...,...,...,...,...,...,...,...
4795,7,18,9,용산구,2024-07-16 09:00:00,293613,여름,출근시간,0,106
4796,7,18,9,은평구,2024-07-18 09:00:00,382284,여름,출근시간,0,2
4797,7,18,9,종로구,2024-07-18 09:00:00,330689,여름,출근시간,0,13
4798,7,18,9,중구,2024-07-18 09:00:00,389344,여름,출근시간,0,8


In [14]:
weather

,행정구,일시,강수,습도,풍속,기온
0,강남구,2025-07-11T00:00,0.0,71,12.7,23.0
1,강남구,2025-07-11T01:00,0.0,74,11.3,22.1
2,강남구,2025-07-11T02:00,0.0,78,11.0,21.5
3,강남구,2025-07-11T03:00,0.0,79,11.4,21.2
4,강남구,2025-07-11T04:00,0.0,82,10.6,20.7
...,...,...,...,...,...,...
4795,중랑구,2025-07-18T19:00,2.1,87,6.4,25.4
4796,중랑구,2025-07-18T20:00,2.1,88,6.3,25.3
4797,중랑구,2025-07-18T21:00,2.1,88,6.2,25.2
4798,중랑구,2025-07-18T22:00,0.0,73,3.3,27.4


In [ ]:
weather_df = weather
weather_df['일시'] = pd.to_datetime(weather_df['일시'])
weather_df['연도'] = weather_df['일시'].dt.year.astype(str)
weather_df['월'] = weather_df['일시'].dt.month.astype(str)
weather_df['일'] = weather_df['일시'].dt.day.astype(str)
weather_df['시'] = weather_df['일시'].dt.hour.astype(str)
weather_df.drop(['일시'],axis=1, inplace=True)
df_with_weather = pd.merge(check,weather_df, on=['행정구','월','일','시'])

,행정구,강수,습도,풍속,기온,연도,월,일,시
0,강남구,0.0,71,12.7,23.0,2025,7,11,0
1,강남구,0.0,74,11.3,22.1,2025,7,11,1
2,강남구,0.0,78,11.0,21.5,2025,7,11,2
3,강남구,0.0,79,11.4,21.2,2025,7,11,3
4,강남구,0.0,82,10.6,20.7,2025,7,11,4
...,...,...,...,...,...,...,...,...,...
4795,중랑구,2.1,87,6.4,25.4,2025,7,18,19
4796,중랑구,2.1,88,6.3,25.3,2025,7,18,20
4797,중랑구,2.1,88,6.2,25.2,2025,7,18,21
4798,중랑구,0.0,73,3.3,27.4,2025,7,18,22


In [19]:
df_with_weather = pd.merge(check,weather_df, on=['행정구','월','일','시'])

In [ ]:
check = fill_missing_timeseries()# _M2 결과
weather = get_hourly_weather_seoul_openmeteo()
check.drop(['연도','강수','습도','풍속','기온'],axis=1, inplace=True)
weather_df = weather
weather_df['일시'] = pd.to_datetime(weather_df['일시'])
weather_df['연도'] = weather_df['일시'].dt.year.astype(str)
weather_df['월'] = weather_df['일시'].dt.month.astype(str)
weather_df['일'] = weather_df['일시'].dt.day.astype(str)
weather_df['시'] = weather_df['일시'].dt.hour.astype(str)
weather_df.drop(['일시'],axis=1, inplace=True)
df_with_weather = pd.merge(check,weather_df, on=['행정구','월','일','시'])

,월,일,시,행정구,일시,총생활인구수_1년전,계절,출퇴근시간여부,주말구분,대여량_1년전,강수,습도,풍속,기온,연도
0,7,11,0,강남구,2024-07-11 00:00:00,701439,여름,기타,0,130,0.0,71,12.7,23.0,2025
1,7,11,0,강동구,2024-07-11 00:00:00,535574,여름,기타,0,126,0.0,69,16.0,23.5,2025
2,7,11,0,강북구,2024-07-11 00:00:00,285612,여름,기타,0,68,0.0,69,12.6,22.8,2025
3,7,11,0,강서구,2024-07-11 00:00:00,546611,여름,기타,0,292,0.0,64,12.6,24.4,2025
4,7,11,0,관악구,2024-07-11 00:00:00,495542,여름,기타,0,117,0.0,66,9.7,23.9,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,7,18,9,용산구,2024-07-16 09:00:00,293613,여름,출근시간,0,106,0.0,63,9.2,30.0,2025
4796,7,18,9,은평구,2024-07-18 09:00:00,382284,여름,출근시간,0,2,0.0,71,6.2,29.4,2025
4797,7,18,9,종로구,2024-07-18 09:00:00,330689,여름,출근시간,0,13,0.0,63,10.5,30.2,2025
4798,7,18,9,중구,2024-07-18 09:00:00,389344,여름,출근시간,0,8,0.0,63,10.5,30.3,2025


In [21]:
import pymysql
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta
import pandas as pd
from itertools import product

def DBMS_last_year_one_week() :
    '''내일을 기준으로 작년 1주일치 생활인구수와 대여량을 DataFrame으로 반환하는 함수'''
    load_dotenv()
    host = os.getenv('host')
    user = os.getenv('user')
    password = os.getenv('password')
    database = os.getenv('database')
    port = int(os.getenv('port', 3306))

    # 현재 시각
    now = datetime.now()

    # 1주일치 시간 list
    this_week = [now + timedelta(days=i) for i in range(8)]

    # 1년 전 1주일치 데이터
    dt_1y = [day.replace(year=day.year - 1) for day in this_week]

    total_last_week = pd.DataFrame(columns = ['연도','월','일','시','행정구','총생활인구수_1년전','대여량_1년전'])
    # total_last_week = pd.DataFrame(columns = ['year','month','day','hour','district','total_population','rental_count'])
    for data in dt_1y :
        query =f"""
            SELECT *
            FROM basic_data
            WHERE year = {data.year}
                AND month = {data.month}
                AND day = {data.day}
                """

        try:
            connection = pymysql.connect(
                host=host,
                user=user,
                password=password,
                database=database,
                port=port,
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            )
            with connection.cursor() as cursor:
                cursor.execute(query)
                data = cursor.fetchall()
                whole_data = pd.DataFrame(data)
                using_data = whole_data[['year','month','day','hour','district','total_population','rental_count']]
                using_data.columns = total_last_week.columns
                total_last_week = pd.concat([total_last_week, using_data],axis=0)
        except Exception as e:
            print('에러 발생:', e)
        finally:
            try:
                connection.close()
            except:
                pass
    return total_last_week # 일주일 치 연도, 월, 일, 시, 행정구, 총생활인구수_1년전, 대여량_1년전

data = DBMS_last_year_one_week()

def fill_missing_timeseries(data=data, days=8, key_cols=None, num_cols=None):
    """
    data: DataFrame, 기준이 되는 원본 데이터 (key_cols 모두 포함)
    days: int, 몇 일간의 데이터를 보장할지 (기본값 8일)
    key_cols: list, 시계열 Key가 되는 컬럼 (예: ['월','일','시','행정구'])
    num_cols: list, 결측값 채울 대상 컬럼. None이면 모든 컬럼 채움

    반환값: 누락 구간이 보간된, 중복 없는 DataFrame
    """
    if key_cols is None:
        key_cols = ['월', '일', '시', '행정구']
    if num_cols is None:
        num_cols = [col for col in data.columns if col not in key_cols]
    for col in key_cols:
        data[col] = data[col].astype(str)
    
    # 1. 전체 조합 만들기
    now = datetime.now().date()
    dates = pd.date_range(now, now + timedelta(days=days-1))   # 8일
    hours = list(range(24))
    gus = data['행정구'].unique().tolist()

    full = []
    for dt in dates:
        for h in hours:
            for g in gus:
                full.append({
                    '월': str(dt.month),
                    '일': str(dt.day),
                    '시': str(h),
                    '행정구': str(g)
                })
    full_df = pd.DataFrame(full)

    # 2. 누락행 탐색 및 합치기
    merged = full_df.merge(data, on=key_cols, how='left', indicator=True)
    missing = merged[merged['_merge']=='left_only']
    df_all = pd.concat([data, missing.drop('_merge', axis=1)], ignore_index=True)
    df_all = df_all.sort_values(key_cols).reset_index(drop=True)

    # 3. ffill, bfill 결측치 채우기 (numeric 및 object 모두 적용 가능)
    for col in num_cols:
        df_all[col] = df_all.groupby(['행정구', '시'])[col].ffill()
        df_all[col] = df_all.groupby(['행정구', '시'])[col].bfill()

    # 4. 중복제거 및 정렬
    df_all = df_all.drop_duplicates(subset=key_cols, keep='first')
    df_all = df_all.sort_values(key_cols).reset_index(drop=True)
    return df_all
import pandas as pd
import numpy as np
import joblib
import holidays
import tensorflow as tf
from datetime import datetime, timedelta
from weather import get_hourly_weather_seoul_openmeteo
from DBMS_lastYear import DBMS_last_year_one_week,fill_missing_timeseries
import matplotlib.pyplot as plt
plt.rc('font', family = "Malgun Gothic")

weather = weather
lastyear_data = fill_missing_timeseries()
le = joblib.load('../../Predict_models/M2/ENCODER_SCALER/labelencoders.pkl')
scaler = joblib.load('../../Predict_models/M2/ENCODER_SCALER/scaler.pkl')
feature_cols = joblib.load('../../Predict_models/M2/ENCODER_SCALER/featurecols.pkl')
num_cols = joblib.load('../../Predict_models/M2/ENCODER_SCALER/numcols.pkl')
model = tf.keras.models.load_model("../../Predict_models/M2/best_dnn_model_lr.h5")
gu_weight = pd.read_csv(r"../../Data_Preprocessing/행정구별자전거대여량비중/행정구top5자전거대여량비중.csv")

def predict_bike_demand():
    """
    내일부터 7일간의 자전거 수요를 예측하는 함수, 강남구, 강동구....중구, 중랑구 순으로 0시부터 24시 까지의 데이터를 순서대로 반환
    """

    df_1y = lastyear_data[['월', '일', '시', '행정구', '총생활인구수_1년전', '대여량_1년전']] # 일주일 치 연도, 월, 일, 시, 행정구, 총생활인구수_1년전, 대여량_1년전
    weather_df = weather
    weather_df['일시'] = pd.to_datetime(weather_df['일시'])
    # weather_df['연도'] = weather_df['일시'].dt.year
    weather_df['월'] = weather_df['일시'].dt.month.astype(str)
    weather_df['일'] = weather_df['일시'].dt.day.astype(str)
    weather_df['시'] = weather_df['일시'].dt.hour.astype(str)

    df_total = weather_df.merge(df_1y, on=['행정구','월','일','시'])
    
    kr_holidays = holidays.KR(years=[datetime.now().year, datetime.now().year+1])
    df_total['요일'] = df_total['일시'].dt.weekday
    df_total['주말구분'] = (df_total['요일'] >= 5).astype(int)
    df_total['공휴일'] = df_total['일시'].dt.date.isin(kr_holidays).astype(int)

#     input_df = pd.DataFrame([input_dict])
    df_total['행정구'] = le.transform(df_total['행정구'].values)
    df_total[num_cols] = scaler.transform(df_total[num_cols])
    X_input = df_total[feature_cols].astype('float32').values
    pred = model.predict(X_input,verbose=0).round().astype(np.int32)
    return pred

datas = predict_bike_demand()

def result_df_return(datas=datas) :
    districts = ['강남구',"강동구","강북구", "강서구","관악구","광진구","구로구","금천구","노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구","용산구","은평구","종로구","중구","중랑구"]
    
    base_date = datetime.now()
    num_days = 8
    num_hours = 24
    all_dates = [base_date + timedelta(days=i) for i in range(num_days)]

    date_hour_list = []
    for d in all_dates:
        for h in range(num_hours):
            date_hour_list.append((d.date(), h))

    # datas shape 자동 보정
    datas = np.array(datas).reshape(len(districts), len(date_hour_list))  
    
    df = pd.DataFrame(datas, index=districts, columns=pd.MultiIndex.from_tuples(date_hour_list, names=['날짜', '시']))

    df_long = df.stack(level=[0, 1]).reset_index()
    df_long.columns = ['행정구', '날짜', '시', '예측값']
    df_long['일시'] = pd.to_datetime(df_long['날짜'].astype(str) + ' ' + df_long['시'].astype(str) + ':00:00')
    df_long['연도'] = df_long['일시'].dt.year.astype(str)
    df_long['월'] = df_long['일시'].dt.month.astype(str)
    df_long['일'] = df_long['일시'].dt.day.astype(str)
    df_long['일시']=df_long['일시'].astype(str)
#     print(df_long.head())
#     print(df_long.shape)  # (3600, 4) => 현재는 7일치로, 4200,4
    return df_long

df = result_df_return()

def check_station_top_five(gu) :
    ''' 행정구를 입력받아 그 행정구의 수요가 높은 5개 역의 비중에 따라 예상 대여량을 그래프로 반환 '''
    
    gu_weight['weight'] = gu_weight['weight'] / 100
    weight_stations = gu_weight[gu_weight['gu'] == gu]
    now = datetime.now().date()
    pred = df
    need_pred = pred[pred['행정구']==gu]
#     selected_gu = gu_weight[gu_weight['gu']==gu][['station','weight']] 
    result_pred = need_pred[need_pred['날짜']==now]
    result = []

    for _, row in result_pred.iterrows():
        for _, srow in weight_stations.iterrows():
            result.append({
                '날짜': row['날짜'],
                '시': row['시'],
                '행정구': row['행정구'],
                'station': srow['station'],
                '예상대여량': round(row['예측값'] * srow['weight'], 2)
            })

    df_station = pd.DataFrame(result)
    station_list = df_station['station'].unique().tolist()
    
    for station in station_list:
        data = df_station[df_station['station']==station]
        plt.figure(figsize=(12, 6))  # 크기 조정

        # Bar chart
        plt.bar(data['시'], data['예상대여량'], color='skyblue', alpha=0.7, label='대여량(예측)')

        # Line chart
        plt.plot(data['시'], data['예상대여량'], c='red', lw=3, marker='o', markersize=8, label='추이')

        # x축 시각화
        plt.xticks([i for i in range(24)], [f'{i}시' for i in range(24)], fontsize=11, rotation=0)
        plt.xlabel('시간(시)', fontsize=14, fontweight='bold')
        plt.ylabel('예상 대여량', fontsize=14, fontweight='bold')

        plt.ylim(0, data['예상대여량'].max() * 1.15)

        # 최대값 라벨 표시
        max_idx = data['예상대여량'].idxmax()
        max_x = data.loc[max_idx, '시']
        max_y = data.loc[max_idx, '예상대여량']
        plt.annotate(f'{max_y:,.0f}', (max_x, max_y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=11, color='red', fontweight='bold')

        plt.title(f'{station}', fontsize=18, fontweight='bold')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.savefig(fname = f'project_root/static/{station}_result.png', dpi=200, bbox_inches='tight')

    return df_station

if __name__ == "__main__" :
    model1_result = result_df_return()
    print(model1_result)
    # check_station_top_five("강서구")

c:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Admin\anaconda3\envs\ml-dl-nlp\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


KeyError: '일시'